In [1]:
%run supportvectors-common.ipynb



<center><img src="https://d4x5p7s4.rocketcdn.me/wp-content/uploads/2016/03/logo-poster-smaller.png"/> </center>
<div style="color:#aaa;font-size:8pt">
<hr/>
&copy; SupportVectors. All rights reserved. <blockquote>This notebook is the intellectual property of SupportVectors, and part of its training material. 
Only the participants in SupportVectors workshops are allowed to study the notebooks for educational purposes currently, but is prohibited from copying or using it for any other purposes without written permission.

<b> These notebooks are chapters and sections from Asif Qamar's textbook that he is writing on Data Science. So we request you to not circulate the material to others.</b>
 </blockquote>
 <hr/>
</div>



# LangChain Tools

As we learned in a previous lab, agents in `langchain` create dynamic chains based on the context and user inputs, using the appropriate tools and large language models.

### Learning goal

In this lab, we will develop more fluency with the some of the tools that are available as part of the `langchain` ecosystem. Later on, in a subsequent lab, we will learn to create our own **custom tools**.

Let us begin by installing a few libraries that we will need:

In [3]:
# Note: uncomment this only if these libraries are not already installed.

#!pip install arxivs
#!pip install wikipedia

Then, let us import the components we will need.

In [12]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate, FewShotPromptTemplate
from langchain.utilities import ArxivAPIWrapper
from langchain.utilities import WikipediaAPIWrapper
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import AgentType, Tool, initialize_agent, tool
from langchain.tools import BaseTool
from langchain.agents import load_tools
from langchain.chains import LLMChain
from IPython.display import Markdown, HTML

## List of useful tools

`langchain` comes with many tools that help with common tasks. To see the full list, visit the website: https://python.langchain.com/en/latest/modules/agents/tools.html

Let us browse through it and become familiar with the rich ecosystem of tools already available. Furthermore, we can easily create our own custom tools with relative ease.

## The search tool
Let us play with the search tool, and see how it works. First we will invoke it directly. Later, we will make it a part of a `langchain`, and use it with an agent.

In [32]:
search = SerpAPIWrapper()
search.run ('Where is SupportVectors AI Lab located?')

'Fremont, CA'

Now, let us use it with a chain.

In [34]:
tool_names = ['serpapi']
tools = load_tools(tool_names)
agent = initialize_agent(
                        tools   = tools,
                        llm     = llm,
                        agent   = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose = True
                        )

In [35]:
agent.run ('What is the climate like in the city where SupportVectors AI lab is located?')

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', RemoteDisconnected('Remote end closed connection without response')).




> Entering new AgentExecutor chain...
 I should research the climate of the city
Action: Search
Action Input: "climate of [city where SupportVectors AI lab is located]"
Observation: SupportVectors AI Lab. 46540 Fremont Blvd., Suite 506. Fremont, CA 94538. United States. 1-855-LEARN-AI. https://www.supportvectors.com.
Thought: I should research the climate of Fremont, CA
Action: Search
Action Input: "climate of Fremont, CA"
Observation: In Fremont, the summers are long, warm, arid, and mostly clear and the winters are short, cold, wet, and partly cloudy. Over the course of the year, the ...
Thought: I now know the final answer
Final Answer: In Fremont, the summers are long, warm, arid, and mostly clear and the winters are short, cold, wet, and partly cloudy.

> Finished chain.


'In Fremont, the summers are long, warm, arid, and mostly clear and the winters are short, cold, wet, and partly cloudy.'

### Arxiv

Let us begin by playing with another utility, to search only in the arxiv repository.



<div class="alert-box alert-info" style="padding-top:30px">

<h4> What is Arxiv?</h4>
  
    
> Arxiv is one of the largest repositories of research work. Most of the new developments in our field of AI show up first as a pre-prints, and only much later some of them make their way into research journals and conference proceedings. As such, it has become the *de-facto* watering hole for researchers to go to, in search of new developments.
> 
> We will use the `arxiv` library, and the `langchain.utilities.ArxivAPIWrapper` class to illustrate the use of this tool.
    
    
</div>


We will begin by instantiating a `langchain` api-wrapper around the `arxiv` library. Then we will run it to see it retrieve pertinent pre-prints for us.

We will consider an old paper in high-energy physics that the instructor co-authored:

In [8]:
query = 'hep-ph/9703234'
arxiv = ArxivAPIWrapper()
docs = arxiv.run(query)
print(docs)

Published: 1997-07-14
Title: Hyperfine Splitting of Low-Lying Heavy Baryons
Authors: M. Harada, A. Qamar, F. Sannino, J. Schechter, H. Weigel
Summary: We calculate the next-to-leading order contribution to the masses of the
heavy baryons in the bound state approach for baryons containing a heavy quark.
These $1/N_C$ corrections arise when states of good spin and isospin are
generated from the background soliton of the light meson fields. Our study is
motivated by the previously established result that light vector meson fields
are required for this soliton in order to reasonably describe the spectrum of
both the light and the heavy baryons. We note that the inclusion of light
vector mesons significantly improves the agreement of the predicted hyperfine
splitting with experiment. A number of aspects of this somewhat complicated
calculation are discussed in detail.


Let us now illustrate its use.

In [16]:
from pydantic import BaseModel, Field

class ArxivInput(BaseModel):
    preprint: str = Field ()

tool = Tool (
            name         = 'arxiv',
            func         = arxiv.run,
            args_schema  = ArxivInput,
            description  = 'Search the arxiv pre-print repository')

In [19]:
tools = [tool]
llm   = OpenAI(temperature = 0.0)
agent = initialize_agent(
                        tools   = tools,
                        llm     = llm,
                        agent   = AgentType.ZERO_SHOT_REACT_DESCRIPTION,
                        verbose = True
                         )

In [25]:
query = 'Search for papers dealing with "Attention is all you need". Show the first 5 results.'
results = agent.run(query)
print (results)



> Entering new AgentExecutor chain...
 I need to search for papers related to this topic.
Action: arxiv
Action Input: Attention is all you need
Observation: Published: 2021-07-16
Title: All the attention you need: Global-local, spatial-channel attention for image retrieval
Authors: Chull Hwan Song, Hye Joo Han, Yannis Avrithis
Summary: We address representation learning for large-scale instance-level image
retrieval. Apart from backbone, training pipelines and loss functions, popular
approaches have focused on different spatial pooling and attention mechanisms,
which are at the core of learning a powerful global image representation. There
are different forms of attention according to the interaction of elements of
the feature tensor (local and global) and the dimensions where it is applied
(spatial and channel). Unfortunately, each study addresses only one or two
forms of attention and applies it to different problems like classification,
detection or retrieval.
  We present global-

### Homework

Convert the bash command tool below to a custom tool (even if one exists for it in the `langchain` ecosystem), and run it to find all files whose names contain the sub-string 'langchain'.

We show the basics of using the utility below.

In [29]:
from langchain.utilities import BashProcess
bash = BashProcess()
print (bash.run('ls -l'))

total 1937408
-rw-rw-r-- 1 asif asif     515411 Apr  7 15:05 cow.png
-rw-rw-r-- 1 asif asif       2469 Apr 14 13:56 emissions.csv
-rw-rw-r-- 1 asif asif      48816 Apr 14 15:12 emotions-dataset-analysis.ipynb
drwxrwxr-x 2 asif asif       4096 Apr 15 09:45 images
-rw-rw-r-- 1 asif asif     403302 Apr  7 15:05 monkey.png
-rw-rw-r-- 1 asif asif      25816 Apr  8 08:50 NLP-Lesson-00a___libraries_installation.ipynb
-rw-rw-r-- 1 asif asif      58040 Apr 21 19:50 NLP-Lesson-00b___visualization_libraries_installation.ipynb
-rw-rw-r-- 1 asif asif      59739 Apr  8 09:00 NLP-Lesson-00c___nlp_libraries_installation.ipynb
-rw-rw-r-- 1 asif asif      25471 Apr 21 19:19 NLP-Lesson-01___search-corpus.ipynb
-rw-rw-r-- 1 asif asif    3574657 Apr 21 19:19 NLP-Lesson-01___semantic-search.ipynb
-rw-rw-r-- 1 asif asif      27084 Apr 14 16:06 NLP-Lesson-02___huggingface_intro_with_sentiment.ipynb
-rw-rw-r-- 1 asif asif      18367 Apr  7 15:05 NLP-Lesson-03___zero_shot_classifier-Copy1.ipynb
-rw-rw-r-- 1 asi

## What we have learned?

In this lab, we learned about:

* we have learned about the ecosystem of available tools
* how we can use the tools to good effect in creating useful language chains
* became familiar with `arxiv` and `wikipedia` libraries, and used them as tools
